In [38]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from delta.tables import *

StatementMeta(, 4bfb9ae1-3d9f-4e6d-a780-7f594c8860b3, 40, Finished, Available)

In [39]:
startyear = 2013
endyear = 2025

StatementMeta(, 4bfb9ae1-3d9f-4e6d-a780-7f594c8860b3, 41, Finished, Available)

In [40]:
lakehousePath = "abfss://85bfc254-9abf-46cc-b1fe-943ec35b3460@msit-onelake.dfs.fabric.microsoft.com/c02dea28-20ca-432b-b6e8-39d0be76f540"
deltaTablePath = f"{lakehousePath}/Tables/Calendar" 

StatementMeta(, 4bfb9ae1-3d9f-4e6d-a780-7f594c8860b3, 42, Finished, Available)

In [41]:
spark = SparkSession.builder.appName("Calendar").getOrCreate()

# Set the time parser policy to LEGACY
# spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Create a DataFrame with a range of dates
dates = spark.range(
    (datetime(endyear, 12, 31) - datetime(startyear, 1, 1)).days + 1
).select(
    (date_add(lit(f"{startyear}-01-01"), col("id").cast("int"))).alias("date")
)

StatementMeta(, 4bfb9ae1-3d9f-4e6d-a780-7f594c8860b3, 43, Finished, Available)

In [42]:
# Select the desired columns
calendardf = dates.select(
    "date",
    dayofmonth("date").alias("daynum"),
    dayofweek("date").alias("dayofweeknum"),
    # date_format("date", "e").alias("dayofweeknum"),
    date_format("date", "EEEE").alias("dayofweekname"),
    month("date").alias("monthnum"),
    date_format("date", "MMMM").alias("monthname"),
    quarter("date").alias("quarternum"),
    concat(lit("Q"), quarter("date")).alias("quartername"),
    year("date").alias("year")
)

StatementMeta(, 4bfb9ae1-3d9f-4e6d-a780-7f594c8860b3, 44, Finished, Available)

In [43]:
# Show the resulting DataFrame
# calendardf.show()

StatementMeta(, 4bfb9ae1-3d9f-4e6d-a780-7f594c8860b3, 45, Finished, Available)

In [44]:
calendardf.write.format("delta").mode("overwrite").save(deltaTablePath)

StatementMeta(, 4bfb9ae1-3d9f-4e6d-a780-7f594c8860b3, 46, Finished, Available)